In [7]:
import csv
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

filename_aws = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\aws\\csv\\aws-analysis.csv"
filename_azure = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\azure\\csv\\azure-analysis.csv"
filename_google = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google-analysis.csv"
filename_aws_tfsec = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\aws\\csv\\aws-analysis-tfsec.csv"
filename_azure_tfsec = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\azure\\csv\\azure-analysis-tfsec.csv"
filename_google_tfsec = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google-analysis-tfsec.csv"

df1 = pd.read_csv(filename_aws)
df2 = pd.read_csv(filename_azure)
df3 = pd.read_csv(filename_google)
df4 = pd.read_csv(filename_aws_tfsec)
df5 = pd.read_csv(filename_azure_tfsec)
df6 = pd.read_csv(filename_google_tfsec)

# Sample DataFrame
data = pd.concat([df1, df2, df3, df4, df5, df6])

# Create DataFrame
df = pd.DataFrame(data)

# Function to map passed rate to categories
def map_passed_rate(passed_rate):
    if passed_rate < 0.2:
        return "LOW_RATE"
    elif passed_rate < 0.4:
        return "LOW_RATE"
    elif passed_rate < 0.6:
        return "MEDIUM_RATE"
    elif passed_rate < 0.8:
        return "HIGH_RATE"
    else:
        return "HIGH_RATE"

    # Function to map other continuous variables to categories based on min-max normalization
def map_to_categories(value, min_val, max_val, category):
    if value < min_val + 0.1 * (max_val - min_val):
        return "VERY_LOW_"+category
    elif value < min_val + 0.2 * (max_val - min_val):
        return "LOW_"+category
    elif value < min_val + 0.3 * (max_val - min_val):
        return "MODERATE_"+category
    elif value < min_val + 0.4 * (max_val - min_val):
        return "BELOW_AVERAGE_"+category
    elif value < min_val + 0.5 * (max_val - min_val):
        return "AVERAGE_"+category
    elif value < min_val + 0.6 * (max_val - min_val):
        return "ABOVE_AVERAGE_"+category
    elif value < min_val + 0.7 * (max_val - min_val):
        return "HIGH_"+category
    elif value < min_val + 0.8 * (max_val - min_val):
        return "VERY_HIGH_"+category
    elif value < min_val + 0.9 * (max_val - min_val):
        return "EXTREMELY_HIGH_"+category
    else:
        return "EXCEPTIONALLY_HIGH_"+category

# Transforming passed rate
df['passed_rate_category'] = df['stars_passed'].apply(map_passed_rate)

# Transforming stars, forks, and contributors
for col in ['stars', 'forks', 'commits', 'commiters']:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col + '_category'] = df[col].apply(lambda x: map_to_categories(x, min_val, max_val, col))

# Print the transformed DataFrame
#print(df.columns)

print(df[['passed_rate_category', 'stars_category', 'forks_category', 'commits_category', 'commiters_category']])

aux = df[['passed_rate_category', 'stars_category', 'forks_category', 'commits_category', 'commiters_category']]

transactions = aux.values

print(transactions)

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_selected = pd.DataFrame(te_ary, columns=te.columns_)

# Generate frequent itemsets with minimum support of 0.3
frequent_itemsets = apriori(df_selected, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift")

# Print the rules
for index, rule in rules.iterrows():
    antecedents = ', '.join(rule['antecedents'])
    consequents = ', '.join(rule['consequents'])
    print(f"Rule: {antecedents} -> {consequents}")
    print(f"Support: {rule['support']}")
    print(f"Confidence: {rule['confidence']}")
    print(f"Lift: {rule['lift']}")
    print()
    

filename = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\rq3\\report-all-tools.csv"

with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["antecedents", "consequents", "Support", "Confidence", "Lift"])
        for index, rule in rules.iterrows():
            antecedents = ', '.join(rule['antecedents'])
            consequents = ', '.join(rule['consequents'])
            csv_writer.writerow([antecedents, consequents, rule['support'], rule['confidence'], rule['lift']])
            

    passed_rate_category        stars_category       forks_category  \
0            MEDIUM_RATE  EXTREMELY_HIGH_stars  BELOW_AVERAGE_forks   
1               LOW_RATE  EXTREMELY_HIGH_stars  BELOW_AVERAGE_forks   
2              HIGH_RATE         AVERAGE_stars       MODERATE_forks   
3            MEDIUM_RATE         AVERAGE_stars  BELOW_AVERAGE_forks   
4              HIGH_RATE   BELOW_AVERAGE_stars        AVERAGE_forks   
..                   ...                   ...                  ...   
206            HIGH_RATE        VERY_LOW_stars       VERY_LOW_forks   
207          MEDIUM_RATE        VERY_LOW_stars       VERY_LOW_forks   
208          MEDIUM_RATE        VERY_LOW_stars       VERY_LOW_forks   
209          MEDIUM_RATE        VERY_LOW_stars       VERY_LOW_forks   
210          MEDIUM_RATE        VERY_LOW_stars       VERY_LOW_forks   

          commits_category       commiters_category  
0    BELOW_AVERAGE_commits        AVERAGE_commiters  
1          AVERAGE_commits  ABOVE_AVERA